In [1]:
%pip install -r ../requirements.txt
%pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# Setup SK

In [2]:
import os
import semantic_kernel as sk
import semantic_kernel.connectors.ai.open_ai as skaoai

kernel = sk.Kernel()

deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

print("Using deployment: ", deployment)
print("Using endpoint: ", endpoint)
masked_api_key = api_key[:3] + '*' * (len(api_key) - 3)
print(f"Using api_key: ", masked_api_key)

kernel.add_service(skaoai.AzureChatCompletion(deployment,endpoint,api_key=api_key))

Using deployment:  Chat
Using endpoint:  https://cajetaoai.openai.azure.com/
Using api_key:  09e*****************************


# Setup Sample Input data

In [6]:
hpi_input = """
Sky Ryder is a 27 y.o. female

Chief Complaint
Patient presents with
•	Establish Care
•	Physical
 	 	With annual labs 
 
 
Pt is here to establish care
Reviewed previous PCP notes and labs
 
Pt states she's been having chronic pain of right knee for many years. She had surgery in 2016 and continues to have pain. She would like referral to ortho
 
Doing well on celexa daily
 
She is currently on semaglutide through weight loss clinic and doing well. 
 
Previous labs noted low HDL and elevated CRP
 
Thyroid Disorder - was getting medication through IHC but states they never monitored lab
 She is seen today for follow up of hypothyroidism. Current symptoms include: denies fatigue, weight changes, heat/cold intolerance, bowel/skin changes or CVS symptoms. Denies swallowing difficulty, voice change or masses noted in the neck.  Thyroid medication is taken daily with 45-60 minutes separating ingestion of food or other medications/supplements. 
This is a chronic problem. Pertinent negatives include no chest pain, no abdominal pain and no shortness of breath. Nothing aggravates the symptoms. Relieved by: medication. 
"""

previous_labs_input = """
Normal 5/2023 labs
"""

assessment_plan_input = """
ASSESSMENT and PLAN: 

ICD-10-CM	ICD-9-CM	 
1.	Encounter for routine adult health examination without abnormal findings 	Z00.00	V70.0	CBC WITHOUT DIFFERENTIAL
 	 	 	COMPREHENSIVE METABOLIC PANEL
 	 	 	COMPREHENSIVE METABOLIC PANEL
 	 	 	CBC WITHOUT DIFFERENTIAL
 
2.	Vitamin D deficiency 	E55.9	268.9	VITAMIN D 25 HYDROXY
 	 	 	VITAMIN D 25 HYDROXY
 
3.	Severe obesity (BMI 35.0-39.9) with comorbidity 	E66.01	278.01	 
 
4.	Acquired hypothyroidism 	E03.9	244.9	TSH
 	 	 	T3 FREE
 	 	 	T4 FREE
 	 	 	T4 FREE
 	 	 	T3 FREE
 	 	 	TSH
 
5.	Low HDL (under 40) 	E78.6	272.5	LIPID PANEL
 	 	 	LIPID PANEL
 
6.	Elevated C-reactive protein (CRP) 	R79.82	790.95	 
 
7.	Prediabetes 	R73.03	790.29	HEMOGLOBIN A1C
 	 	 	INSULIN LEVEL
 	 	 	INSULIN LEVEL
 	 	 	HEMOGLOBIN A1C
 
8.	Chronic pain of right knee 	M25.561	719.46	AMB REFERRAL TO ORTHOPEDIC SURGERY
 	G89.29	338.29	 
 
 
 
Labs today
Referral to ortho for knee. Alternate Ibuprofen 400mg with Tylenol 500mg every 4 hours for pain and inflammation. 
Medications reviewed, no refills needed
Advised to schedule with GYN for PAP
"""

lab_results_input = """
HDL: 40 mg/dL
CHOLESTEROL: 191 mg/dL
TRIGLYCERIDE: 84 mg/dL
LDL CALCULATED: 133 mg/dL (calc)
CHOL/HDL RATIO: 4.8 (calc)
TOTAL NON-HDL CHOLESTEROL (LDL+VLDL): 151 mg/dL (calc)

"""

input_text = """Lab Results: {lab_results_input} 
Last patient encounter notes follow: 
{hpi_input}
{previous_labs_input}
{assessment_plan_input}
""".format(lab_results_input=lab_results_input, hpi_input=hpi_input, previous_labs_input=previous_labs_input, assessment_plan_input=assessment_plan_input)

print(input_text)



Lab Results: 
HDL: 40 mg/dL
CHOLESTEROL: 191 mg/dL
TRIGLYCERIDE: 84 mg/dL
LDL CALCULATED: 133 mg/dL (calc)
CHOL/HDL RATIO: 4.8 (calc)
TOTAL NON-HDL CHOLESTEROL (LDL+VLDL): 151 mg/dL (calc)

 
Last patient encounter notes follow: 

Sky Ryder is a 27 y.o. female

Chief Complaint
Patient presents with
•	Establish Care
•	Physical
 	 	With annual labs 
 
 
Pt is here to establish care
Reviewed previous PCP notes and labs
 
Pt states she's been having chronic pain of right knee for many years. She had surgery in 2016 and continues to have pain. She would like referral to ortho
 
Doing well on celexa daily
 
She is currently on semaglutide through weight loss clinic and doing well. 
 
Previous labs noted low HDL and elevated CRP
 
Thyroid Disorder - was getting medication through IHC but states they never monitored lab
 She is seen today for follow up of hypothyroidism. Current symptoms include: denies fatigue, weight changes, heat/cold intolerance, bowel/skin changes or CVS symptoms. Denies 

# Setup & Run Prompt Function Inline

Set this up as seen in example here: 
[Semantic-kernel/python/notebooks/03-prompt-function-inline.ipynb - Running Prompt Functions Inline](https://github.com/microsoft/semantic-kernel/blob/9b8a218f5b6df9dcc72d69d989aff9b904cdecf0/python/notebooks/03-prompt-function-inline.ipynb)

Run the previous block first to set sample input. Adjust the prompt here and run test different prompts

In [4]:
from semantic_kernel.prompt_template.input_variable import InputVariable

# initialize prompt to be set in next code block
prompt = """Intructions: You are a clinician sending lab results in a message to a patient after reviewing your notes from the last patient visit.
Always address the patient directly and professionally
Always explain what the results mean in a way that is easy for the patient to understand
Always keep the message kind and concise

{{$input}}
"""

execution_settings = skaoai.OpenAIPromptExecutionSettings(
        ai_model_id=deployment,
        max_tokens=3000,
        temperature=0.7,
    )

prompt_template_config = sk.PromptTemplateConfig(
    template=prompt,
    name="summarize",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The user input", is_required=True),
    ],
    execution_settings=execution_settings,
)

summarize = kernel.create_function_from_prompt(
    function_name="summarizeFunc",
    plugin_name="summarizePlugin",
    prompt_template_config=prompt_template_config,
)

summary = await kernel.invoke(summarize, sk.KernelArguments(input=input_text))

print(summary)

Dear Sky Ryder,

I hope this message finds you well. I wanted to follow up with your lab results from our last visit. Your HDL (good cholesterol) levels are low at 40 mg/dL and your total cholesterol is elevated at 191 mg/dL. Your LDL (bad cholesterol) levels are also elevated at 133 mg/dL. Your cholesterol to HDL ratio is 4.8, which is higher than the recommended range. Your total non-HDL cholesterol (LDL+VLDL) is 151 mg/dL. 

These results indicate that we need to work on reducing your cholesterol levels. I recommend making lifestyle changes such as increasing physical activity and following a heart-healthy diet. I also suggest discussing medication options with me during our next visit. 

Please let me know if you have any questions or concerns. 

Best regards,
[Your Name]
